## Optimisation des hyperparamètres Nu et Gamma

Paramètre nu: limite supérieure sur la fraction d'erreurs de marge et une limite inférieure de la fraction des vecteurs supports p/rap au nombre total d'exemples d'entrainement.
Si on le fixe à 0.95, on est assuré d'avoir au plus 95% de nos exemples d'entrainement mal classés mais au moins 95% des exemples seront des vecteurs supports.

In [1]:
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn import svm

In [2]:
data_train = pd.read_hdf('train.hdf5')
data_valid = pd.read_hdf('validation.hdf5')

data_train['y_train'] = 1
x_train = data_train.iloc[:,0:61440]
y_train = data_train.iloc[:,61440:61441]

### Aperçu de la démarche à adopter pour le GridSearchCV

In [ ]:
nus = [0.001, 0.01, 0.1, 1]
gammas = [0.001, 0.01, 0.1, 1]
# grille de paramètres
tuned_parameters = {'kernel' : ['rbf'], 'gamma' : gammas, 'nu': nus}

scores = ['precision', 'recall']
for score in scores:
    # initialiser une recherche sur grille
    clf = GridSearchCV(svm.OneClassSVM(), tuned_parameters, cv=10,
                           scoring='%s_macro' % score, return_train_score=True)
    # faire tourner la recherche sur grille
    clf.fit(X_train, y_train)
    # grid.best_params_, grid.best_score_
    resultDf = pd.DataFrame(clf.cv_results_)

In [ ]:
tuned_parameters = {'kernel' : ['rbf'], 'gamma' : [0.001, 0.01, 0.1, 1], 'nu': [0.25, 0.5, 0.75, 0.95]}
clf = GridSearchCV(svm.OneClassSVM(), tuned_parameters, cv=5,scoring='f1',return_train_score=True)
clf.fit(x_train, y_train)
resultDf = pd.DataFrame(clf.cv_results_)

In [ ]:
pred_anomaly = clf.predict(data_valid)

In [ ]:
from collections import Counter
Counter(pred_anomaly)

Autre idée: reprendre le script et boucler sur les deux paramètres à la fois

In [ ]:
tb_count_moins1 = pd.DataFrame(index = ['Count outliers'])
tb_count_1 = pd.DataFrame(index = ['Count inliers'])

seq_gamma = [0.01, 0.001, 0.0001]
seq_nu = [0.25, 0.5, 0.75, 0.95]
for gamma_i in seq_gamma:
    for nu_i in seq_nu:
        tb_pred = pd.DataFrame()
        SVM_oneclass = svm.OneClassSVM(kernel="rbf", gamma=gamma_i, nu=nu_i)
        SVM_oneclass.fit(data_train) 
        pred_anomaly = SVM_oneclass.predict(data_valid)
        
#         tb_count_1[gamma_i] = [int(Counter(pred_anomaly)[1])]
#         tb_count_moins1[gamma_i] = [int(Counter(pred_anomaly)[-1])]
#     tb_count_gamma = pd.concat([tb_count_1, tb_count_moins1])
#     tb_count_gamma

### Exportation du fichier y_test_template à soumettre

In [ ]:
pred_anomaly = clf.predict(data_valid)
from collections import Counter
Counter(pred_anomaly)

pred_anomaly2 = pd.DataFrame(pred_anomaly)
pred_anomaly2 = pred_anomaly2.replace(1, 0)
pred_anomaly2 = pred_anomaly2.replace(-1, 1)
data = {'seqID': np.arange(0,len(data_valid)), 'anomaly': pred_anomaly2[0]}
y_test_template = pd.DataFrame(data)

anomalies = y_test_template[y_test_template.anomaly == 1]
y_anomaly = pd.DataFrame(anomalies)
y_anomaly.head()